In [1]:
import pandas as pd

In [2]:
import tensorflow
tensorflow.keras.__version__

'2.2.4-tf'

# Read the CSV and Perform Basic Data Cleaning

In [3]:
df = pd.read_csv("../Resources/exoplanet_data.csv")

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Display dataframe
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select features and label

In [4]:
# Set features as X value and label as y value
X = df.drop(columns=['koi_disposition'])
y = df['koi_disposition']

# Create a Train Test Split

Use `koi_disposition` for the y values

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [6]:
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from tensorflow.keras.utils import to_categorical

# Scale your data
X_minmax = MinMaxScaler().fit(X_train)

X_train_scaled = X_minmax.transform(X_train)
X_test_scaled = X_minmax.transform(X_test)

In [7]:
# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

# Create Model

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Create a Neural Network model here
model = Sequential()

# Add layers to model
model.add(Dense(units=100, activation='relu', input_dim=len(X.columns)))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=3, activation='softmax'))

In [9]:
# Compile and fit the model: https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping
model.compile(optimizer='adam',
                   loss='categorical_crossentropy',
                   metrics=['accuracy'])

callback = tensorflow.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

model.fit(
    X_train_scaled,
    y_train_categorical,
    callbacks=[callback],
    batch_size=1,
    epochs=100,
    shuffle=True,
    verbose=2,
)

Train on 5243 samples
Epoch 1/100
5243/5243 - 17s - loss: 0.4128 - accuracy: 0.7789
Epoch 2/100
5243/5243 - 11s - loss: 0.3534 - accuracy: 0.8203
Epoch 3/100
5243/5243 - 9s - loss: 0.3431 - accuracy: 0.8278
Epoch 4/100
5243/5243 - 10s - loss: 0.3304 - accuracy: 0.8396
Epoch 5/100
5243/5243 - 9s - loss: 0.3247 - accuracy: 0.8461
Epoch 6/100
5243/5243 - 9s - loss: 0.3233 - accuracy: 0.8499
Epoch 7/100
5243/5243 - 10s - loss: 0.3177 - accuracy: 0.8533
Epoch 8/100
5243/5243 - 11s - loss: 0.3086 - accuracy: 0.8558
Epoch 9/100
5243/5243 - 11s - loss: 0.3081 - accuracy: 0.8627
Epoch 10/100
5243/5243 - 10s - loss: 0.3022 - accuracy: 0.8625
Epoch 11/100
5243/5243 - 11s - loss: 0.2996 - accuracy: 0.8642
Epoch 12/100
5243/5243 - 9s - loss: 0.2941 - accuracy: 0.8741
Epoch 13/100
5243/5243 - 9s - loss: 0.2968 - accuracy: 0.8703
Epoch 14/100
5243/5243 - 9s - loss: 0.2940 - accuracy: 0.8720
Epoch 15/100
5243/5243 - 10s - loss: 0.2936 - accuracy: 0.8728
Epoch 16/100
5243/5243 - 10s - loss: 0.2892 - ac

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               4100      
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 303       
Total params: 14,503
Trainable params: 14,503
Non-trainable params: 0
_________________________________________________________________


In [11]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(f"Deep Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

1748/1 - 0s - loss: 0.3060 - accuracy: 0.8804
Deep Neural Network - Loss: 0.2999497754350134, Accuracy: 0.8804348111152649


In [12]:
encoded_predictions = model.predict_classes(X_test_scaled)
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [13]:
prediction_df = pd.DataFrame({
    'Actual': list(y_test),
    'Prediction': prediction_labels,
})

prediction_df.head(10)

,Actual,Prediction
0,CANDIDATE,CANDIDATE
1,FALSE POSITIVE,FALSE POSITIVE
2,FALSE POSITIVE,FALSE POSITIVE
3,CANDIDATE,CANDIDATE
4,FALSE POSITIVE,FALSE POSITIVE
5,CONFIRMED,CONFIRMED
6,FALSE POSITIVE,FALSE POSITIVE
7,FALSE POSITIVE,FALSE POSITIVE
8,CONFIRMED,CANDIDATE
9,CONFIRMED,FALSE POSITIVE


# Save the Model

In [14]:
# Save model
filename = '../Models/deep_learning.h5'
model.save(filename)